<a href="https://colab.research.google.com/github/xteox017/Sintesis-de-reuniones-con-IA-Equipo-31-TEC/blob/main/Avance6_Equipo31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Maestría en Inteligencia Artificial Aplicada

PROYECTO INTEGRADOR

**Avance 6. Conclusiones claves**

Equipo 31

JUAN CARLOS VILLAMIL ROJAS A01794003

MATEO CRUZ LANCHERO A01793882

ANDREA MARGARITA OSORIO GONZÁLEZ A01104776

9/6/2024

**Avance #6** (Trabajo en progreso, modelo final aún no definido)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install moviepy transformers
!pip install openai==0.28
!pip install transformers
!pip install bert-extractive-summarizer
!pip install rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00


In [ ]:
from moviepy.editor import VideoFileClip
from google.colab import drive
drive.mount('/content/drive')

# Función para extraer audio de un video
def extract_audio(video_path):
    video = VideoFileClip(video_path)
    audio_path = "extracted_audio.wav"
    video.audio.write_audiofile(audio_path)
    return audio_path

video_path = '/content/drive/My Drive/Videos/sistema_financiero _ Hernan Casciari _ TED.mp4'
audio_path = extract_audio(video_path)
print("Audio extraído:", audio_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MoviePy - Writing audio in extracted_audio.wav


MoviePy - Done.
Audio extraído: extracted_audio.wav


In [ ]:
!pip install pydub
from pydub import AudioSegment
import math
import os

# Función para dividir el audio en segmentos más pequeños
def split_audio(audio_path, max_segment_size_bytes=25000000):  # 25 MB
    audio = AudioSegment.from_wav(audio_path)
    duration_ms = len(audio)
    segments = []
    segment_length_ms = 0

    # Calcula la longitud de los segmentos basándose en el tamaño máximo permitido
    for i in range(duration_ms):
        if (audio[i:i+1000].frame_rate * audio[i:i+1000].sample_width * audio[i:i+1000].channels * (i+1000) / 8) <= max_segment_size_bytes:
            segment_length_ms += 1000
        else:
            break

    for start in range(0, duration_ms, segment_length_ms):
        end = min(start + segment_length_ms, duration_ms)
        segment = audio[start:end]
        segment_path = f"segment_{start//1000}_{end//1000}.wav"
        segment.export(segment_path, format="wav")
        segments.append(segment_path)

    return segments

segments = split_audio(audio_path)
print("Segmentos de audio generados:", segments)

# Verifica el tamaño de cada segmento
for segment in segments:
    print(f"Tamaño de {segment}: {os.path.getsize(segment)} bytes")


Segmentos de audio generados: ['segment_0_134.wav', 'segment_134_268.wav', 'segment_268_402.wav', 'segment_402_536.wav', 'segment_536_670.wav', 'segment_670_804.wav', 'segment_804_938.wav', 'segment_938_1072.wav', 'segment_1072_1206.wav', 'segment_1206_1340.wav', 'segment_1340_1351.wav']
Tamaño de segment_0_134.wav: 23637644 bytes
Tamaño de segment_134_268.wav: 23637644 bytes
Tamaño de segment_268_402.wav: 23637644 bytes
Tamaño de segment_402_536.wav: 23637644 bytes
Tamaño de segment_536_670.wav: 23637644 bytes
Tamaño de segment_670_804.wav: 23637644 bytes
Tamaño de segment_804_938.wav: 23637644 bytes
Tamaño de segment_938_1072.wav: 23637644 bytes
Tamaño de segment_1072_1206.wav: 23637644 bytes
Tamaño de segment_1206_1340.wav: 23637644 bytes
Tamaño de segment_1340_1351.wav: 2048048 bytes


In [ ]:
from google.colab import files

# Subir el archivo de la clave de API
uploaded = files.upload()

# Leer la clave de API
api_key = None
for fn in uploaded.keys():
    with open(fn) as f:
        api_key = f.read().strip()


KeyboardInterrupt: 

In [ ]:
import openai

# Configurar la clave de API de OpenAI
openai.api_key = api_key

# Función para transcribir un segmento de audio a texto utilizando OpenAI
def transcribe_segment_openai(segment_path):
    with open(segment_path, "rb") as audio_file:
        response = openai.Audio.transcribe(
            model="whisper-1",
            file=audio_file
        )
    return response["text"]

# Función para transcribir todos los segmentos de audio y concatenar los resultados
def transcribe_audio_segments(segment_paths):
    full_transcript = ""
    for segment_path in segment_paths:
        transcript = transcribe_segment_openai(segment_path)
        full_transcript += transcript + " "
    return full_transcript.strip()

transcript = transcribe_audio_segments(segments)
print("Transcripción del audio:", transcript)



In [ ]:
# Función para guardar la transcripción en un archivo .txt
def save_transcript(transcript, video_path):
    # Obtener el nombre base del archivo de video
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    # Crear el nombre del archivo de transcripción
    transcript_filename = f"{video_name}_transcript.txt"

    with open(transcript_filename, "w") as file:
        file.write(transcript)

    print(f"Transcripción guardada en: {transcript_filename}")

# Guardar la transcripción antes de la limpieza
save_transcript(transcript, video_path)


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

nltk.download('punkt')
nltk.download('stopwords')

# Función para limpiar el texto
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'[%s]' % re.escape(r'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Función para preprocesar el texto
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()

    text = clean_text(text)
    words = word_tokenize(text)
    words = [ps.stem(word) for word in words if word not in stop_words]

    return ' '.join(words)

cleaned_transcript = preprocess_text(transcript)
print("Texto preprocesado:", cleaned_transcript)


In [ ]:
from transformers import pipeline

# Cargar el modelo de resumen especificando el nombre del modelo
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")

# Función para dividir el texto en partes más pequeñas
def split_text(text, max_length=512):  # Ajustar longitud a 512 tokens
    sentences = text.split('. ')
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence.split())
        if current_length + sentence_length > max_length:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentence]
            current_length = sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length

    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')

    return chunks


In [ ]:
import time

# Función para resumir cada parte del texto
def summarize_text_chunks(chunks):
    summaries = []
    total_time = 0

    for chunk in chunks:
        input_length = len(chunk.split())
        max_length = min(150, input_length)  # Ajustar max_length a la longitud del fragmento
        min_length = max(25, max_length // 2)  # Ajustar min_length basado en max_length

        if input_length > 5:  # Solo intentar resumir si el fragmento tiene más de 5 palabras
            start_time = time.time()
            try:
                summary = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)
                summaries.append(summary[0]['summary_text'])
            except Exception as e:
                print(f"Error al resumir el fragmento: {e}")
                summaries.append(chunk)  # Añadir el fragmento original si hay un error
            end_time = time.time()
            elapsed_time = end_time - start_time
            total_time += elapsed_time
        else:
            summaries.append(chunk)  # Si el fragmento es muy corto, usarlo como está

    return ' '.join(summaries), total_time


In [ ]:
chunks = split_text(cleaned_transcript)
summary_transformer, time_transformer = summarize_text_chunks(chunks)
print("Resumen (Transformers):", summary_transformer)
print("Tiempo de procesamiento (Transformers):", time_transformer)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

# Función para generar resumen con TF-IDF
def summarize_with_tfidf(text):
    start_time = time.time()

    sentences = text.split('. ')
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    scores = sim_matrix.sum(axis=1)
    ranked_sentences = [sentences[i] for i in np.argsort(scores, axis=0)[::-1]]

    summary = '. '.join(ranked_sentences[:5])  # Tomar las top 5 frases más relevantes
    end_time = time.time()
    elapsed_time = end_time - start_time

    return summary, elapsed_time

summary_tfidf, time_tfidf = summarize_with_tfidf(cleaned_transcript)
print("Resumen (TF-IDF):", summary_tfidf)
print("Tiempo de procesamiento (TF-IDF):", time_tfidf)



In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Cargar el modelo y el tokenizador GPT-2
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained("gpt2")
model_gpt2 = GPT2LMHeadModel.from_pretrained("gpt2")

# Función para generar resumen con GPT-2
def summarize_with_gpt2(text, max_new_tokens=100):
    start_time = time.time()
    inputs = tokenizer_gpt2.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model_gpt2.generate(inputs, max_new_tokens=max_new_tokens, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer_gpt2.decode(summary_ids[0], skip_special_tokens=True)
    end_time = time.time()
    elapsed_time = end_time - start_time
    return summary, elapsed_time



In [ ]:
from summarizer import Summarizer

# Cargar el modelo BERT para resumen extractivo
bert_model = Summarizer()

# Función para generar resumen con BERT Extractive Summarizer con medición de tiempo
def summarize_with_bert(text):
    start_time = time.time()
    try:
        summary = bert_model(text, min_length=60, max_length=500)
    except Exception as e:
        summary = ""
    end_time = time.time()
    elapsed_time = end_time - start_time
    return summary, elapsed_time




In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Función para generar resumen con LSA con medición de tiempo
def summarize_with_lsa(text):
    start_time = time.time()
    sentences = text.split('. ')
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(sentences)
    lsa = TruncatedSVD(n_components=1, n_iter=100)
    lsa.fit(X)
    scores = lsa.components_[0]

    # Asegurarse de que no excedemos la longitud de sentences
    ranked_indices = np.argsort(scores, axis=0)[::-1]
    ranked_sentences = [sentences[i] for i in ranked_indices if i < len(sentences)]

    summary = '. '.join(ranked_sentences[:5])  # Tomar las top 5 frases más relevantes
    end_time = time.time()
    elapsed_time = end_time - start_time
    return summary, elapsed_time




In [ ]:
# Ejecutar la división y el resumen de texto ajustado
chunks = split_text(cleaned_transcript)
summary_transformer, time_transformer = summarize_text_chunks(chunks)
print("Resumen (Transformers):", summary_transformer)
print("Tiempo de procesamiento (Transformers):", time_transformer)

summary_gpt2, time_gpt2 = summarize_with_gpt2(cleaned_transcript)
print("Resumen (GPT-2):", summary_gpt2)

summary_bert, time_bert = summarize_with_bert(cleaned_transcript)
print("Resumen (BERT Extractive):", summary_bert)

summary_lsa, time_lsa = summarize_with_lsa(cleaned_transcript)
print("Resumen (LSA):", summary_lsa)

summary_tfidf, time_tfidf = summarize_with_tfidf(cleaned_transcript)
print("Resumen (TF-IDF):", summary_tfidf)
print("Tiempo de procesamiento (TF-IDF):", time_tfidf)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV

# Crear un conjunto de documentos ficticios
documents = [
    "This is a dummy document. It contains text to be used for testing.",
    "Another dummy document. This one also contains text for testing purposes.",
    "Yet another dummy document. It is used to simulate a larger dataset.",
    "More dummy text. This document is part of the test set.",
    "Final dummy document. This concludes our set of test documents."
]

# Vectorizar las oraciones del conjunto de documentos
vectorizer = TfidfVectorizer()
X_documents = vectorizer.fit_transform(documents)

# Definir un rango de hiperparámetros para LSA
param_grid_lsa = {
    'n_components': [1, 2, 3, 4, 5],
    'n_iter': [100, 200, 300]
}

# Crear una función de evaluación usando GridSearchCV para LSA
def optimize_lsa(X, param_grid):
    lsa = TruncatedSVD()
    grid_search = GridSearchCV(lsa, param_grid, cv=2, scoring='neg_mean_squared_error')  # Ajustar a 2 divisiones
    grid_search.fit(X)
    best_lsa = grid_search.best_estimator_
    return best_lsa

# Optimizar LSA usando el conjunto de documentos
best_lsa = optimize_lsa(X_documents, param_grid_lsa)


In [ ]:
from rouge import Rouge

# Ajustar el límite de recursión
import sys
sys.setrecursionlimit(2000)

# Evaluar métricas ROUGE
rouge = Rouge()

# Asegurarse de que los textos no sean demasiado largos
max_evaluation_length = 512

# Función para evaluar ROUGE de manera segura
def safe_rouge_score(summary, reference, rouge):
    if not summary.strip():
        return {'rouge-1': {'f': 0.0}, 'rouge-2': {'f': 0.0}, 'rouge-l': {'f': 0.0}}
    return rouge.get_scores(summary[:max_evaluation_length], reference[:max_evaluation_length])[0]

scores_transformer = safe_rouge_score(summary_transformer, transcript, rouge)
scores_gpt2 = safe_rouge_score(summary_gpt2, transcript, rouge)
scores_bert = safe_rouge_score(summary_bert, transcript, rouge)
scores_lsa = safe_rouge_score(summary_lsa, transcript, rouge)
scores_tfidf = safe_rouge_score(summary_tfidf, transcript, rouge)

print("Scores (Transformers):", scores_transformer)
print("Scores (GPT-2):", scores_gpt2)
print("Scores (BERT Extractive):", scores_bert)
print("Scores (LSA):", scores_lsa)
print("Scores (TF-IDF):", scores_tfidf)

# Mostrar tiempos de procesamiento
print("Tiempo de procesamiento (Transformers):", time_transformer)
print("Tiempo de procesamiento (GPT-2):", time_gpt2)
print("Tiempo de procesamiento (BERT Extractive):", time_bert)
print("Tiempo de procesamiento (LSA):", time_lsa)
print("Tiempo de procesamiento (TF-IDF):", time_tfidf)


In [ ]:
# Generar las predicciones y evaluar
chunks = split_text(cleaned_transcript)
summary_transformer, time_transformer = summarize_text_chunks(chunks)
summary_gpt2, time_gpt2 = summarize_with_gpt2(cleaned_transcript)
summary_bert, time_bert = summarize_with_bert(cleaned_transcript)
summary_lsa, time_lsa = summarize_with_lsa(cleaned_transcript)
summary_tfidf, time_tfidf = summarize_with_tfidf(cleaned_transcript)

# Evaluar los resúmenes
scores_transformer = safe_rouge_score(summary_transformer, cleaned_transcript, rouge)
scores_gpt2 = safe_rouge_score(summary_gpt2, cleaned_transcript, rouge)
scores_bert = safe_rouge_score(summary_bert, cleaned_transcript, rouge)
scores_lsa = safe_rouge_score(summary_lsa, cleaned_transcript, rouge)
scores_tfidf = safe_rouge_score(summary_tfidf, cleaned_transcript, rouge)

print("Scores (Transformers):", scores_transformer)
print("Scores (GPT-2):", scores_gpt2)
print("Scores (BERT Extractive):", scores_bert)
print("Scores (LSA):", scores_lsa)
print("Scores (TF-IDF):", scores_tfidf)

# Mostrar tiempos de procesamiento
print("Tiempo de procesamiento (Transformers):", time_transformer)
print("Tiempo de procesamiento (GPT-2):", time_gpt2)
print("Tiempo de procesamiento (BERT Extractive):", time_bert)
print("Tiempo de procesamiento (LSA):", time_lsa)
print("Tiempo de procesamiento (TF-IDF):", time_tfidf)




In [ ]:
import pandas as pd

# Crear una tabla con los resultados de las métricas y los tiempos de procesamiento
results = {
    'Model': ['Transformers', 'GPT-2', 'BERT Extractive', 'LSA', 'TF-IDF', 'Stacking', 'Voting'],
    'ROUGE-1 F1': [scores_transformer['rouge-1']['f'], scores_gpt2['rouge-1']['f'], scores_bert['rouge-1']['f'], scores_lsa['rouge-1']['f'], scores_tfidf['rouge-1']['f'], 0.0, 0.0],  # Placeholder para ensamble
    'ROUGE-2 F1': [scores_transformer['rouge-2']['f'], scores_gpt2['rouge-2']['f'], scores_bert['rouge-2']['f'], scores_lsa['rouge-2']['f'], scores_tfidf['rouge-2']['f'], 0.0, 0.0],  # Placeholder para ensamble
    'ROUGE-L F1': [scores_transformer['rouge-l']['f'], scores_gpt2['rouge-l']['f'], scores_bert['rouge-l']['f'], scores_lsa['rouge-l']['f'], scores_tfidf['rouge-l']['f'], 0.0, 0.0],  # Placeholder para ensamble
    'Processing Time (s)': [time_transformer, time_gpt2, time_bert, time_lsa, time_tfidf, 0.0, 0.0]  # Placeholder para ensamble
}

results_df = pd.DataFrame(results)
results_df.sort_values(by='ROUGE-1 F1', ascending=False, inplace=True)
print(results_df)
